In [ ]:
from dotenv import load_dotenv
import os
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.tools import tool
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.agents import initialize_agent, Tool

# ====================================================
# 1. 환경 변수 불러오기
# ====================================================
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print("OPENAI_KEY:", OPENAI_API_KEY[:2])
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
print("TAVILY_KEY:", TAVILY_API_KEY[:4])

# ====================================================
# 2. 로컬 메뉴 데이터 읽기 및 벡터 DB 생성
# ====================================================
MENU_FILE = "../data/cafe_menu_data.txt"
DB_PATH = "../db/cafe_db"

with open(MENU_FILE, "r", encoding="utf-8") as f:
    raw = f.read().strip()

entries = [e.strip() for e in raw.split("\n\n") if e.strip()]

docs = []
for e in entries:
    lines = [l.strip(" •") for l in e.splitlines() if l.strip()]
    meta, content_lines = {}, []

    for ln in lines:
        if ln[0].isdigit() and "." in ln:
            meta["name"] = ln.split(".", 1)[1].strip()
        elif ln.startswith("가격:"):
            meta["price"] = ln.replace("가격:", "").strip()
        elif ln.startswith("주요 원료:"):
            meta["ingredients"] = ln.replace("주요 원료:", "").strip()
        elif ln.startswith("설명:"):
            meta["description"] = ln.replace("설명:", "").strip()
            content_lines.append(meta["description"])
        else:
            content_lines.append(ln)

    content = " ".join(content_lines)
    docs.append(Document(page_content=content, metadata=meta))

print(f"총 {len(docs)}개의 메뉴를 불러왔습니다.")

# 벡터 DB 생성
embeddings = OpenAIEmbeddings()
faiss_index = FAISS.from_documents(docs, embeddings)
faiss_index.save_local(DB_PATH)
print(f"벡터 DB가 {DB_PATH} 경로에 저장되었습니다.")

# ====================================================
# 3. 도구 정의
# ====================================================
# a) Tavily Search Tool
tavily_search_func = TavilySearchResults(api_key=TAVILY_API_KEY)

# b) Wikipedia Summary Tool
wiki_wrapper = WikipediaAPIWrapper(lang="ko", top_k_results=2)

@tool("wiki_summary", return_direct=False)
def wiki_summary(query: str) -> str:
    """위키피디아에서 검색 주제에 대한 요약 정보를 가져옵니다."""
    try:
        return wiki_wrapper.run(query)
    except Exception as e:
        return f"위키 검색 실패: {str(e)}"

# c) Local Cafe Menu Search Tool
@tool("db_search_cafe_func", return_direct=False)
def db_search_cafe_func(query: str) -> str:
    """로컬 카페 메뉴 DB에서 정보를 검색합니다."""
    retriever = faiss_index.as_retriever(search_kwargs={"k": 2})
    results = retriever.get_relevant_documents(query)
    out = []
    for r in results:
        meta = r.metadata
        out.append(
            f"메뉴: {meta.get('name')}, 가격: {meta.get('price')}, 원료: {meta.get('ingredients')}, 설명: {meta.get('description')}"
        )
    return "\n".join(out) if out else "관련 메뉴를 찾지 못했습니다."

# ====================================================
# 4. LLM 및 Agent 초기화
# ====================================================
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

agent_tools = [
    Tool(
        name="wiki_summary",
        func=wiki_summary,
        description="위키피디아에서 주제에 대한 요약 정보를 가져옵니다."
    ),
    Tool(
        name="db_search_cafe_func",
        func=db_search_cafe_func,
        description="로컬 카페 메뉴 DB에서 정보를 검색합니다."
    ),
    Tool(
        name="tavily_search_func",
        func=tavily_search_func.run,
        description="웹 검색을 통해 최신 정보를 가져옵니다."
    )
]

agent = initialize_agent(
    tools=agent_tools,
    llm=llm,
    agent="zero-shot-react-description",  # 도구 호출이 가능한 agent 유형
    verbose=True
)

# ====================================================
# 5. 테스트 실행
# ====================================================
question = "아메리카노의 가격과 특징은 무엇인가요?"
response = agent.run(question)

print("사용자 질문:", question)
print("LLM 응답:", response)


OPENAI_KEY: sk
TAVILY_KEY: tvly
총 10개의 메뉴를 불러왔습니다.
벡터 DB가 ../db/cafe_db 경로에 저장되었습니다.


> Entering new AgentExecutor chain...


C:\Users\csu19\AppData\Local\Temp\ipykernel_26544\1884056222.py:114: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
C:\Users\csu19\AppData\Local\Temp\ipykernel_26544\1884056222.py:125: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(question)


아메리카노의 가격과 특징에 대한 정보를 찾기 위해 카페 메뉴 데이터베이스를 검색해보겠습니다. 
Action: db_search_cafe_func
Action Input: '아메리카노 가격과 특징'
Observation: 메뉴: 아이스 아메리카노, 가격: ₩4,500, 원료: 에스프레소, 차가운 물, 얼음, 설명: 진한 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 아이스 커피입니다. 깔끔하고 시원한 맛이 특징이며, 원두 본연의 풍미를 느낄 수 있습니다. 더운 날씨에 인기가 높습니다.
메뉴: 바닐라 라떼, 가격: ₩6,000, 원료: 에스프레소, 스팀 밀크, 바닐라 시럽, 설명: 카페라떼에 달콤한 바닐라 시럽을 더한 인기 메뉴입니다. 바닐라의 달콤함과 커피의 쌉싸름함이 조화롭게 어우러지며, 휘핑크림 토핑으로 더욱 풍성한 맛을 즐길 수 있습니다.
Thought:아메리카노의 가격과 특징에 대한 정보를 찾았습니다. 아메리카노는 아이스 아메리카노로 제공되며, 가격은 ₩4,500입니다. 이 음료는 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 커피로, 깔끔하고 시원한 맛이 특징입니다. 특히 더운 날씨에 인기가 높으며, 원두 본연의 풍미를 느낄 수 있습니다.

Final Answer: 아메리카노의 가격은 ₩4,500이며, 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 커피로, 깔끔하고 시원한 맛이 특징입니다.

> Finished chain.
사용자 질문: 아메리카노의 가격과 특징은 무엇인가요?
LLM 응답: 아메리카노의 가격은 ₩4,500이며, 에스프레소에 차가운 물과 얼음을 넣어 만든 시원한 커피로, 깔끔하고 시원한 맛이 특징입니다.
